In [11]:
from git import Repo
from langchain_community.document_loaders.generic import GenericLoader
from langchain_community.document_loaders.parsers import LanguageParser
from langchain_text_splitters import Language,RecursiveCharacterTextSplitter
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import Chroma

In [ ]:
repo = Repo.clone_from("https://github.com/RocketChat/Rocket.Chat.git", to_path="Rocket.Chat", branch="develop")

In [4]:
loader = GenericLoader.from_filesystem(
    "Rocket.Chat/",
    glob="**/*",
    suffixes=[".ts",".tsx"],
    parser=LanguageParser(language=Language.PYTHON, parser_threshold=500),
)
documents = loader.load()

6266

In [7]:
python_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.PYTHON, chunk_size=2000, chunk_overlap=200
)
chunks = python_splitter.split_documents(documents)

11706

In [13]:
embedding=OllamaEmbeddings(model="nomic-embed-text")

In [14]:
vectorDb=Chroma.from_documents(
    documents=chunks,
    embedding=embedding,
    persist_directory="./chroma_db"
)
vectorDb.persist()